In [1]:
# Import the libraries needed for the project
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # transform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries downloaded!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [2]:
# Download the Toronto table from Wikipedia and transform into a dataframe

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url, header=0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# Remove rows with 'Not assigned' values since we don't need them for the task at hand
df = df[df.Borough != 'Not assigned']
df.head(20)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


# Dataframe with Toronto postal codes, boroughs, neighborhoods 
### Data downloaded from Wikipedia

In [4]:
# Reset the count of the index to reflect the actual number of rows
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# Check the size of the dataframe df
df.shape

(103, 3)

In [6]:
# Download geospatial data to retrieve latitude and longitude coordinates of the neighborhoods
!wget -q -O 'geo_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [7]:
# Open the CSV data and transform into dataframe format
with open('geo_data.csv') as geo_data:
    df_ll = pd.read_csv(geo_data)

In [8]:
df_ll.sort_values(by='Postal Code')

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
#Get a preview of the dataframe df_ll
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Updated dataframe with Toronto postal codes, boroughs, neighborhoods, latitude and longitude coordinates

In [10]:
new_df = pd.merge(df, df_ll, how='outer', on=['Postal Code'])
new_df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [11]:
new_df.shape

(103, 5)

In [12]:
# Get the coordinates of Toronto
address_toronto = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address_toronto)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario, are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario, are 43.6534817, -79.3839347.


In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [14]:
for lat, lng, borough, neighborhood in zip(
        new_df['Latitude'], 
        new_df['Longitude'], 
        new_df['Neighbourhood'],
        new_df['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
 
map_toronto

# Studying the borough Scarborough

In [15]:
# Check out the borough Scarborough - the easternmost borough of Toronto
scarborough_df = new_df[new_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_df.rename(columns={'Postal Code': 'Postal Code', 'Borough': 'Borough', 'Neighbourhood': 'Neighborhood', 'Latitude': 'Latitude', 'Longitude': 'Longitude'}, inplace=True)
scarborough_df.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [16]:
print('There are {} neighborhoods in Scarborough.'.format(scarborough_df['Borough'].count()))

There are 17 neighborhoods in Scarborough.


In [17]:
address_scarborough = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address_scarborough)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Scarborough are 43.773077, -79.257774.


In [18]:
# Create map of the neighborhoods of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_df['Latitude'], scarborough_df['Longitude'], scarborough_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [19]:
CLIENT_ID = 'QQK5BE4GNRNEI04RLQBVW0XK5V33XNPXL1JTOTGIU2NH24IW' # your Foursquare ID
CLIENT_SECRET = '2TSXP51FGZZBLJJU3VADO0ZLCKAFDLDXSTYM01LFAAPRKRFW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: QQK5BE4GNRNEI04RLQBVW0XK5V33XNPXL1JTOTGIU2NH24IW
CLIENT_SECRET:2TSXP51FGZZBLJJU3VADO0ZLCKAFDLDXSTYM01LFAAPRKRFW


In [20]:
# Explore the neighborhoods of Scarborough
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
scarborough_venues = getNearbyVenues(names=scarborough_df['Neighborhood'],
                                   latitudes=scarborough_df['Latitude'],
                                   longitudes=scarborough_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [22]:
print(scarborough_venues.shape)
scarborough_venues.head()

(92, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [23]:
# Check out how many of the nearby venues are in each neighborhood of Scarborough.
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [24]:
print('There are {} unique categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 52 unique categories.


##### There's 1 neighborhood missing in scarborough_venues: Upper Rouge. Now there are only 16 instead of 17 neighborhoods in the dataframe.
###### The reason is that there are no venues found in Upper Rouge. Looking at the map, it looks like a hiking trail / park area. 

In [25]:
# Study the rest of the neighborhoods

# one hot encoding of the venues' category names
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [26]:
scarborough_onehot.shape

(92, 53)

In [27]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.25,0.250000,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.111111,0.111111,0.00,0.000000,0.111111,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.111111,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.000000,0.00,0.071429,0.000000,0.0,0.000000,0.000000,0.071429,0.071429,0.071429,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.071429,0.071429,0.00,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.0,0.071429,0.000000,0.0,0.142857,0.142857,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.071429,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.5,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.4,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.000000,0.200000
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.111111,0.0,0.111111,0.000000,0.00,0.00,0.000000,0.000,0.111111,0.000,0.000000,0.0,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.000000,0.125000,0.0,0.125000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.125000,0.000000,0.000000

In [28]:
scarborough_grouped.shape

(16, 53)

In [29]:
# Print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0     Thai Restaurant  0.11
1              Bakery  0.11
2                Bank  0.11
3  Athletics & Sports  0.11
4    Hakka Restaurant  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                venue  freq
0         Pizza Place  0.14
1            Pharmacy  0.14
2  Italian Restaurant  0.07
3        Noodle House  0.07
4   Convenience Store  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1                Motel   0

In [30]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [158]:
# Discover the top 10 venues' category types of each neighborhood of Scarborough
import numpy as np 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(17)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Cedarbrae,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Bank,Bakery,Bar
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Convenience Store,Gas Station,Noodle House,Italian Restaurant,Intersection,Fried Chicken Joint,Bank,Thai Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Athletics & Sports,Auto Garage,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Metro Station,Ice Cream Shop,Intersection,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
7,"Guildwood, Morningside, West Hill",Medical Center,Electronics Store,Restaurant,Rental Car Location,Bank,Breakfast Spot,Intersection,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
8,"Kennedy Park, Ionview, East Birchmount Park",Chinese Restaurant,Bus Station,Department Store,Coffee Shop,Hobby Shop,Bank,Convenience Store,General Entertainment,Gas Station,Fried Chicken Joint
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop


# K-Means Clustering

In [159]:
# Cluster the neighborhoods
# Set number of clusters to 5
kclusters = 5
 
scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)
 
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)
 
# check cluster labels generated for each row in the dataframe
kmeans_labels = kmeans.labels_
kmeans_labels

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 2, 0, 1, 1, 1], dtype=int32)

In [160]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_labels)

neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,1,Cedarbrae,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Bank,Bakery,Bar
3,1,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Convenience Store,Gas Station,Noodle House,Italian Restaurant,Intersection,Fried Chicken Joint,Bank,Thai Restaurant
4,3,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Athletics & Sports,Auto Garage,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [161]:
# merge scarborough_merged with neighborhoods_venues_sorted to add latitude/longitude for each neighborhood
scarborough_merged = pd.merge(scarborough_df, neighborhoods_venues_sorted, how='outer', on=['Neighborhood'])

scarborough_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Medical Center,Electronics Store,Restaurant,Rental Car Location,Bank,Breakfast Spot,Intersection,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Bank,Bakery,Bar
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Chinese Restaurant,Bus Station,Department Store,Coffee Shop,Hobby Shop,Bank,Convenience Store,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Metro Station,Ice Cream Shop,Intersection,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3.0,American Restaurant,Motel,Athletics & Sports,Auto Garage,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [162]:
scarborough_merged.shape

(17, 16)

In [163]:
scarborough_merged.dropna(inplace=True)
scarborough_merged.shape

(16, 16)

In [174]:
scarborough_merged_final = scarborough_merged.astype({'Cluster Labels': 'int32'}, copy=True, errors='ignore', inplace=True)
scarborough_merged_final

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Medical Center,Electronics Store,Restaurant,Rental Car Location,Bank,Breakfast Spot,Intersection,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Bank,Bakery,Bar
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Chinese Restaurant,Bus Station,Department Store,Coffee Shop,Hobby Shop,Bank,Convenience Store,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Metro Station,Ice Cream Shop,Intersection,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3,American Restaurant,Motel,Athletics & Sports,Auto Garage,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [175]:
# Create map of the neighborhood clusters of Scarborough
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged_final['Latitude'], scarborough_merged_final['Longitude'], scarborough_merged_final['Neighborhood'], scarborough_merged_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(map_clusters)
       
map_clusters

# Analyze the clusters

In [182]:
scarborough_cleanedup = scarborough_merged_final.sort_values(by=['Cluster Labels'])
scarborough_cleanedup

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Medical Center,Electronics Store,Restaurant,Rental Car Location,Bank,Breakfast Spot,Intersection,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Bank,Bakery,Bar
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Chinese Restaurant,Bus Station,Department Store,Coffee Shop,Hobby Shop,Bank,Convenience Store,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Metro Station,Ice Cream Shop,Intersection,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
10,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
11,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,1,Vietnamese Restaurant,Smoke Shop,Sandwich Place,Auto Garage,Bakery,Breakfast Spot,Middle Eastern Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Analysis: For clusters 0, 2, 3, 4, there is only one neighborhood comprising them. Cluster 1, on the other hand, comprises of 12 neighborhoods.

##### Get more information about each cluster:

In [188]:
# Check out cluster 1
scarborough_merged_final.loc[scarborough_merged_final['Cluster Labels'] == 0, scarborough_merged_final.columns[[2] + list(range(5, scarborough_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop


#### Distinguishing characteristics of cluster 1
1. Comprising of only 1 neighborhood: Scarborough Village. It seems to be a residential area. 
2. Top 3 venue types are: Playgrounds, Vietnamese and Chinese restaurants.
3. Has various types of stores, including electronics, discount, department and cosmetic shops.
-----

In [189]:
# Check out cluster 2
scarborough_merged_final.loc[scarborough_merged_final['Cluster Labels'] == 1, scarborough_merged_final.columns[[2] + list(range(5, scarborough_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",1,Medical Center,Electronics Store,Restaurant,Rental Car Location,Bank,Breakfast Spot,Intersection,Mexican Restaurant,Vietnamese Restaurant,Fast Food Restaurant
3,Woburn,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Cedarbrae,1,Hakka Restaurant,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Lounge,Thai Restaurant,Athletics & Sports,Bank,Bakery,Bar
6,"Kennedy Park, Ionview, East Birchmount Park",1,Chinese Restaurant,Bus Station,Department Store,Coffee Shop,Hobby Shop,Bank,Convenience Store,General Entertainment,Gas Station,Fried Chicken Joint
7,"Golden Mile, Clairlea, Oakridge",1,Bakery,Bus Line,Metro Station,Ice Cream Shop,Intersection,Bus Station,Park,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
9,"Birch Cliff, Cliffside West",1,General Entertainment,Skating Rink,College Stadium,Café,Chinese Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
10,"Dorset Park, Wexford Heights, Scarborough Town...",1,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
11,"Wexford, Maryvale",1,Vietnamese Restaurant,Smoke Shop,Sandwich Place,Auto Garage,Bakery,Breakfast Spot,Middle Eastern Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant
12,Agincourt,1,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
13,"Clarks Corners, Tam O'Shanter, Sullivan",1,Pizza Place,Pharmacy,Convenience Store,Gas Station,Noodle House,Italian Restaurant,Intersection,Fried Chicken Joint,Bank,Thai Restaurant


#### Distinguishing characteristics of cluster 2
1. Comprises of 12 neighborhoods, a big majority of the borough.
2. The Scarborough City Centre, where the one of the major business districts outside Downtown Toronto, is located here. Hospitals, banks, department stores can be found in this cluster.  
2. Most common venues among the 12 neighborhoods are mixed, but there is a wide variety of restaurants, stores, entertainment places (e.g. skating rink, lounge), and transportation modes (e.g. metro, bus stations).
3. Since many of the residents of Scarborough are of mixed backgrounds / nationalities, this cluster is likely where most of them live.
-----

In [190]:
# Check out cluster 3
scarborough_merged_final.loc[scarborough_merged_final['Cluster Labels'] == 2, scarborough_merged_final.columns[[2] + list(range(5, scarborough_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",2,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


#### Distinguishing characteristics of cluster 3
1. Found on the eastern side of the Scarborough borough.
2. Comprised of the 1 neighborhood, Rouge Hill, Port Union, Highland Creek.
3. This cluster is very urbanized with the top venues being bars, Vietnamese restaurants and coffee shops. 
-----

In [191]:
# Check out cluster 4
scarborough_merged_final.loc[scarborough_merged_final['Cluster Labels'] == 3, scarborough_merged_final.columns[[2] + list(range(5, scarborough_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffside, Cliffcrest, Scarborough Village West",3,American Restaurant,Motel,Athletics & Sports,Auto Garage,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


#### Distinguishing characteristics of cluster 4
1. Comprised of 1 neighborhood, Cliffside, Cliffcrest, Scarborough Village West.
2. As this neighborhood is located near the port and marina, where locals and tourists would visit and enjoy seaside and water-based activities, the top venues include motels, athletics and sports areas. 
-----

In [192]:
# Check out cluster 5
scarborough_merged_final.loc[scarborough_merged_final['Cluster Labels'] == 4, scarborough_merged_final.columns[[2] + list(range(5, scarborough_merged_final.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",4,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop


#### Distinguishing characteristics of cluster 5
1. Found on the northeast side of the borough, near the Rouge National Urban Park.
2. Comprising of 1 neighborhood: Malvern, Rouge. This is most likely a rural / residential area.
3. Top 3 venues are restaurants: fast food, Vietnamese and Thai.
----

## Scarborough is an interesting borough of Toronto, a large business district surrounded by greenery, cliffs, rivers, and Lake Ontario. Businesses looking to invest will find an attractive area for trade with many established businesses already located here. It's also a good place to live in since nature is nearby and its multi-cultural character delivers plenty of opportunities for new experiences. 